# TME 8 : Mini-projet Detection de motifs


## Partie D : Recheche de pattern (motifs) en utilisant les suffix trees

Nous allons utiliser l'algorithme suffix-tree pour la recherche rapide et éfficace de motifs. Un suffix-tree est construit à partir d'un jeux de séquences, ensuite nous pouvons rechercher le motif en temps O(k) où k est la longueur du motif.

1\. Nous allons réutiliser les fonctions du TME6 pour générer ``t`` séquences artificielles de taille ``n``, et implanter dans chaque séquence un motif de taille ``k`` à des positions aléatoires avec ``v`` substitutions choisies aléatoirement. Cependant, les ``t`` séquences artificielles initiales (sans implantation) ainsi que le motif initial (sans variation/mutation) doivent être générées une seule fois. Ensuite, selon chaque question, nous introduisons des différentes variation au motif initial et les implantons dans les séquences initiales afin de générer des nouveau jeux de données. 

In [1]:
import random
import numpy as np
import copy

nuc = ('A', 'C', 'G', 'T')

k=9 #taille de motif
v=0 #nb de positions variables dans le motif
t=50 #nb de sequences
n=100 #longuer des sequence


1.1\. Generer les séquences artificielles initiales et implanter un motif (sans variation, v=0)

In [2]:
def generateRandomSequence(n, I):
    seq_tmp = ""
    
    if I == True:
        for i in range(n):
            seq_tmp = seq_tmp + random.choice(nuc)
    else:
        for i in range(n):
            seq_tmp = seq_tmp + random.choice(nuc).lower()

    return seq_tmp
    

def generateRandomSequences(n, t):
    sequences = [generateRandomSequence(n, True) for i in range(t)]
   
    return sequences


def implantMotifVar(k, v, t, n):
    sequences = generateRandomSequences(n, t)
    sequences_ini = copy.deepcopy(sequences)
    motif = generateRandomSequence(k, True)
    motif_ini = copy.deepcopy(motif)
    print("Motif : ", motif)
    
    for i in range(len(sequences)):
        L_seq = list(sequences[i])
        L_motif = list(motif)
        for nb_mutation in range(v):
            index_mutation_motif = random.randint(0,len(motif) - 1)
            L_motif[index_mutation_motif] = random.choice(nuc)
        chn_motif_mute = "".join(L_motif)
        #print(chn_motif_mute)
        L_seq.insert(random.randint(0, len(L_seq) - 1), chn_motif_mute)
        
        sequences[i] = "".join(L_seq)
    return sequences_ini, sequences, motif_ini

sequences_ini, sequences_implanted, fix_motif = implantMotifVar(k, v, t, n)
print ("ADN : ", sequences_implanted)
print("Motif : ", fix_motif)

Motif :  CCTCGATCA
ADN :  ['GAAACTACAGTTAGAGCTACACGTACCACATCCACAGCAGGATACTTTCTTTGTGGGCGAGATCGAAGCTAGGACACCCTCGATCAGATCTCAGGTAGCAATGCGTGCT', 'TCTCAGAATACACCGTCCGTTAGACGTCCTTTGCGGGCGAGGGATGATTCTCCCTCATGACTTTGGTAATCCTGTCGTCGCCCTCGATCAAAATATCGCTCCGGGTAAT', 'AGAACCTCGATCAGTTTGGATTAGTAAGTACCCATAGATCTTCGCTGATCACGGAACTGGCGGAGTTTAAGGTCAAGCCTGGGCCAGGCCATTGGGAAAAGATCACCAT', 'ATCGAGTCCTGCCTCGATCACTCGCTGCCCATATATCCGATCTCCTACTAATGGCGTTGTGACTTGCCTAGTGTGGATAGAGCCACTATGAGCTCAGTATACGCACCTT', 'CCAGTTGCTCCGACACTATCATAATCACCGAATAGGCGATAAGCATGGCGAAGATCGGCGCTTGCGAAAGCAAGCCTCGATCAATTCAACTTTTGTCTCTTCCAACGTT', 'CCGCGAACCGCACTCCGGTGCAGTGTACCGGACAAAGCAGACACAGCGTCCCCCCCTCGATCATAATGGGAAGGTGGAAAGGAATAGGCTGCCCGCAAGCGTATATCTG', 'AAACCTCGATCACAGAGCATGAGTTCCAGCGGGCGGGGCCACCGGATATTTTGAAAGTGTCGTTTCTCACCAAACGTTCGGAAAGCAACGTCGTCGTCGCCGGGCTGTC', 'ACAGAAGCGCTTACATACGTAGACGACCGCGAATACCTCGATCACGGAGAGCGCAGGGTCAGCTACTCTCGTGGCGGCTTTACATCCCAAAGGGGCGAGAGTTTCGCGC', 'CGTCCTTTGGTGAGAATCTGGACTGGCACACCCAATTCTCGGGGATGGTGGATATCGTAGAGGCATCC

2\. Définissez une fonction ``construct_tree`` pour construire un suffix tree à partir des séquences artificielles (après implantation) en utilisant le python package suffix-trees trouvable ici: https://pypi.org/project/suffix-trees/. Tester si votre fonction est capable de trouver le motif sans variation implanté.

In [3]:
from suffix_trees import STree

st = STree.STree("abcdefghab")
print(st.find("def")) # 0
print(st.find_all("ab")) # [0, 8]

a = ["xxxabcxxx", "adsaabcaa", "ytysabcre", "qqqabcqwa", "aaabcaaaa"]
st = STree.STree(a)
print(st.lcs()) # "abc"
print(st.find_all("qqqabcqwa"))
print(a[30//9])


def construct_tree(sequences):
    return STree.STree(sequences)

tree = construct_tree(sequences_implanted)
print(tree.find_all(fix_motif))


3
{0, 8}
abc
{30}
qqqabcqwa
{514, 1924, 3204, 1797, 3721, 2832, 1170, 2707, 663, 5015, 3611, 2974, 2595, 805, 5159, 4653, 3502, 946, 4276, 4923, 3387, 4157, 1597, 191, 3136, 2368, 2498, 4802, 4043, 77, 1231, 1743, 3923, 341, 4311, 1371, 604, 5471, 224, 1503, 4579, 5351, 2925, 4462, 5231, 3823, 2292, 2041, 1021, 2174}


3\. Avant de chercher les motifs, implémentez ou reutilisez les fonctions pour générer tous les motifs (k-mer) possibles de taille k, en éliminant les motifs peu complexe pour éviter les calculs inutiles.

In [4]:
import itertools
def generateKmers(alphabet, k):#avec k = 9 et len(alphabet) = 4, il y a plus de 200 000 combinaison possible !
    validKmers = []
    nuc_str = "".join(alphabet)
    kmers = itertools.product(nuc_str, repeat = k)
    validKmers = list(kmers)
    
    return validKmers


def tousLesKmers(ADN, k): #+ optimal pour la question 8, car ca evitera beaucoup de calculs inutiles
    """
    List(String) -> List(String)
    Rend la liste de tous les k-mers present dans chaque element de ADN
    """
    List_des_Kmer = []
    for sequence in ADN:
        for i in range(len(sequence)-k+1):
            seq_i = ""
            for j in range(k):
                seq_i = seq_i + sequence[i+j]
            if not seq_i in List_des_Kmer:
                List_des_Kmer.append(seq_i)
    return List_des_Kmer

def removeLowComplexe(motifs, minrep):
    validMotif = []
    for mot in motifs:
        if mot.count('A') < minrep and mot.count('C') < minrep and mot.count('G') < minrep and mot.count('T') < minrep:
            validMotif.append("".join(mot))
    return validMotif

def removeLowComplexePair(motifs):
    validMotif = []
    
    for cle in motifs:
        list_tmp = []
        list_tmp.append(cle[0])
        for i in range(2, len(cle), 2):
            if cle[i] == list_tmp[0]:
                list_tmp.append(cle[i])
        cpt_pos_pair = len(list_tmp)
        list_tmp = []
        list_tmp.append(cle[1])
        for i in range(3, len(cle), 2):
            if cle[i] == list_tmp[0]:
                list_tmp.append(cle[i])
        cpt_pos_impair = len(list_tmp)
        
        if not cpt_pos_pair == ((len(cle) // 2) + 1) and not cpt_pos_impair == (len(cle) // 2):
            validMotif.append(cle)
              
    return validMotif

minrep = 5

kmers = generateKmers(nuc, k)
kmersValid = removeLowComplexe(kmers, minrep)
kmersValid = removeLowComplexePair(kmersValid)

#print(kmersValid)


4\. **Exact matching:** Définissez la fonction ``exact_match`` qui cherche dans le suffix tree tous les motifs possibles (k-mers), générés à la question precedent. La fonction renvoie un dictionnaire qui contient les motifs (keys) et leurs nombre d'occurrence (values). Ce dictionnaire doit être trié par nombre d'occurrences. 

Ensuite, trouvez et affichez les 10 motifs plus fréquents dans notre jeux de données artificiels.

In [5]:
import operator

def exact_match(kmersV, stree):
    match = {}
    for kmer in kmersV:
        res_tmp_tree = stree.find_all(kmer)
        match[kmer] = len(res_tmp_tree)
        
    sorted_match = sorted(match.items(), key=operator.itemgetter(1), reverse = True)
    final_sorted_match = {k:v for k,v in sorted_match}
        
    return final_sorted_match

st = construct_tree(sequences_implanted)
motif_occur_sorted = exact_match(kmersValid, st)
print(list(motif_occur_sorted.items())[:10])  #correct si motif implanté est complexe

[('CCTCGATCA', 50), ('TCCTCGATC', 20), ('CTCGATCAC', 15), ('CTCGATCAG', 13), ('CTCGATCAA', 12), ('ACCTCGATC', 11), ('CTCGATCAT', 10), ('GCCTCGATC', 10), ('ATCCTCGAT', 8), ('TCGATCACA', 6)]


5\. Avez-vous réussi à trouver votre motif initial implanté en séquences? l'algorithme était-il rapide? Faites attention aux valeurs élevées des variable k, t, et n par rapport aux TMEs précedants. Quelle est la complexité de chaque recherche de motif? 




Par rapport aux TMEs précedents, même si les variables k, t et n ont des valeurs plus grandes qu'aux TMEs précedents,
cet algorithme est plus rapide.

Et, oui j'ai réussi à trouver le motif initial implanté.
Par contre, si les motif implanté est un motif PEU COMPLEXE, on ne le retrouvera pas car il ne sera pas dans kmersValid





6\. Introduisez deux variation (v=2) au motif initial. Pour cela avant de chaque implantation, créez d'abord un motif varié (avec v substitutions choisies aléatoirement) à partir du motif initial et puis implantez-le dans une séquence. Repetez pour chaque sequence dans le Jeux de donnée. Il suffit de mettre ``v`` égal ``2`` et réutiliser les fonctions définies à la question 1.

In [6]:
v=2

def variation_motif(motif,v):
    """
    String * int -> String
    Retourne le motif avec v substitutions placé aléatoirement
    """
    motif_varie = copy.deepcopy(motif)
    L_motif = list(motif_varie)
    for nb_mutation in range(v):
        index_mutation_motif = random.randint(0,len(motif) - 1)
        L_motif[index_mutation_motif] = random.choice(nuc)
    
    motif_varie = "".join(L_motif)
    
    return motif_varie

def implantMotifVarie(seqs, v, motif):
    """
    List[String] * int * String -> List[String]
    Retourne la liste des sequences avec les motifs ayant v substitution implanté dans chaque sequence
    """
    sequence_implante = copy.deepcopy(seqs)
    
    for i in range(len(sequence_implante)):
        L_seq = list(sequence_implante[i])
        motif_mute = variation_motif(motif, v)
        L_seq.insert(random.randint(0, len(L_seq) - 1), motif_mute)
        sequence_implante[i] = "".join(L_seq)
    
    return sequence_implante
    
    
#print(variation_motif(fix_motif, v))
sequences_implanted_with_var = implantMotifVarie(sequences_ini, v, fix_motif)
print(sequences_implanted_with_var)

kmers2 = tousLesKmers(sequences_implanted_with_var, k) #Pour la question suivante
kmersValid2 = removeLowComplexe(kmers2, minrep)
kmersValid2 = removeLowComplexePair(kmersValid2)

['GAAACTACAGTTAGAGCTACACGTACCACATCCACAGCAGGATACTTTCTTTGACTCAATCATGGGCGAGATCGAAGCTAGGACACGATCTCAGGTAGCAATGCGTGCT', 'TCTCAGAATACACCGTCCGTTAGACGTCCTTTGCGGGCGAGGGATGATTCTCCCTCATGACTTTGGTAATCCTGTCGTCGCAAATATCGCTCCGGCCTCGAGGAGTAAT', 'AGAAGTTTGGATTAGTAAGTACCCATAGATCTTCGCTGATCACGGAACTGGCGGAGTTTAAGGTCAAGCCTGGGCCAGGCCCTTGATCACATTGGGAAAAGATCACCAT', 'ATCGAGTCCTGCTCGCTGCCCATATATCCGATCTCCTACTAATGGCGTTGTGACTTGCCTAGTGTGGATAGAGCCCCTCAACCAACTATGAGCTCAGTATACGCACCTT', 'CCAGTTGCTCCGACACTATCATAATCACCGAATAGGCACTGGATCAGATAAGCATGGCGAAGATCGGCGCTTGCGAAAGCAAGATTCAACTTTTGTCTCTTCCAACGTT', 'CCGCGAACCGCACTCCGGTGCAGTGTACCGGACAAAGCAGACACAGCGTCCCCCTAATGGGAAGGTGGAAAGGAATAGGCTGCCCGCAAGCGTATCCTTGATCGATCTG', 'AAACAGAGCATGAGTTCCAGCGGGCGGGGCCACCGGATATTTTGAAAGTGTCGTTTCTCACCAAACGTTCGGAAAGCAACGTCGTCCCTCGCTCAGTCGCCGGGCTGTC', 'ACAGCCACGATCAAAGCGCTTACATACGTAGACGACCGCGAATACGGAGAGCGCAGGGTCAGCTACTCTCGTGGCGGCTTTACATCCCAAAGGGGCGAGAGTTTCGCGC', 'CGTCCTTTGGTGAGAATCTGGACTGGCACCCTCGATTAACCCAATTCTCGGGGATGGTGGATATCGTAGAGGCATTATCTAAAAGGTCGCCAAC

7\. Construisez le suffix tree à nouveau à partir des nouvelles séquences en utilisant le python package suffix-trees.

In [7]:
tree2 = construct_tree(sequences_implanted_with_var)

8\. **Inexact matching:** 

Définissez fonction ``inexact_match`` qui cherche tous les motifs possibles (k-mers) générés à la question 2 dans le nouveau suffix tree donné (construit à partir des nouvelle séquences qui incluent le motif varié), et renvoie un dictionnaire qui contient les motifs (keys) et les listes de toutes leurs variations (values) ainsi que le meilleur motif variable. Il faut que vous utilisiez la algorithm *seed* pour trouver le motif variable. 

Ensuite, affichez le meilleur motif variable avec toutes son variation dans notre nouveaux jeux de données artificiels.

In [9]:

def hamdist(v, x):
    diffs = 0
    for i in range(len(x)):
        if v[i] != x[i]:
            diffs = diffs + 1
    
    return diffs


def decoupe_en_seed(text, motif, k, d, tree):
    Ns = d + 1
    Ls = k // Ns
    allCandidates = []
    
    for i in range(Ns):
        seed = motif[i*Ls : i*Ls + Ls]
        candidateIndex = tree.find_all(seed)
        for index in candidateIndex:
            if not tree.find(text[index//(n + k + 1)]) == 0: #index//(n + k + 1) = numero de sequence
                candidateText = text[index//(n + k + 1)][(index % (tree.find(text[index//(n + k + 1)])))-i*Ls : (index % (tree.find(text[index//(n + k + 1)])))+k-i*Ls]
                if len(candidateText) == k and hamdist(motif, candidateText) <= d:
                    allCandidates.append(candidateText)
            else:
                candidateText = text[0][index-i*Ls : index+k-i*Ls]
                if len(candidateText) == k and hamdist(motif, candidateText) <= d:
                    allCandidates.append(candidateText)
                
    
    return allCandidates

def inexact_match(kmersV, sequences, stree, v):
    dic_des_var = {}
    
    for kmer in kmersV:
        candi = decoupe_en_seed(sequences, kmer, k, v, stree)
        if kmer not in dic_des_var:
            dic_des_var[kmer] = candi, len(candi)
        else:
            dic_des_var[kmer] = dic_des_var[kmer] + candi, len(dic_des_var[kmer] + candi)
            
        sorted_inexact_match = sorted(dic_des_var.items(), key=lambda x: x[1][1], reverse = True)
        final_sorted_inexact_match = {k:v for k,v in sorted_inexact_match}
                        
    return final_sorted_inexact_match
            
    
        
print("v = ",v)  
print("Nb de kmers complexe dans 'sequences_implanted_with_var' : ",len(kmersValid2))
test3 = inexact_match(kmersValid2, sequences_implanted_with_var, tree2, v)
print(list(test3.items())[0])

mot,(seq_tmp, nbVar) = list(test3.items())[0]

file = open("FichierPourLogo.txt", "w")
for s in seq_tmp:
    file.write(s)
    file.write("\n")
file.close()
    

    

v =  2
Nb de kmers complexe dans 'sequences_implanted_with_var' :  3918
('CCTCGATCA', (['CCTCGCTCA', 'CCTCGTTCA', 'CCTGGATCG', 'CCTCGATTA', 'CCTCCATCA', 'CCTTGATCG', 'CCTCGGTCA', 'CCTCGATCA', 'CCTCTATCA', 'CCTCCATCA', 'CCTCGAGCA', 'CCTCGAGCA', 'CCTCGAGGA', 'CCTCGCTCA', 'CCTCGTTCA', 'CCTTGATCA', 'CCTCGTTCA', 'CCTCGATCA', 'CCTCAATCG', 'CCTCGTGCA', 'CCTCCATCC', 'CCTCGATTA', 'CCTCAACCA', 'CCTCCTTCA', 'CCTCGGTCG', 'CCTCGAACA', 'CCTCGATTA', 'CACCGATCA', 'CGTCGATCT', 'CGTCGATCG', 'CCTCGATTA', 'CCTCGATCA', 'GCGCGATCA', 'CCTCGAGCA', 'CCTCGAGGA', 'CCTCGAGCA', 'TCTCGATCA', 'CCCCGATAA', 'CCACGATCA', 'TCTCGATGA', 'CATCGATCA', 'ACTCGATCA', 'CCTCGATCA', 'CCACGATCA', 'CCGCGATCA', 'CCTCGATTA', 'CCGCGATCA', 'ACTCGATCA', 'CTTCGATGA', 'CCTCGAACA', 'CCTCGATTA', 'CCTCGCTCA', 'CACCGATCA', 'CCTCGTTCA', 'ACTCAATCA', 'CCTCCATCA', 'TCTGGATCA', 'CATCGCTCA', 'CCTCGGTCA', 'CCTCGATCA', 'CCTCTATCA', 'GCGCGATCA', 'CCTCCATCA', 'TCTCGATCA', 'CCACCATCA', 'CCTCGCTCA', 'CCACGATCA', 'CATCGATCA', 'CCTCGTTCA', 'CCTTGATCA', 'C

9\. Créez le motif logo à partir des séquences du meilleur motif variable que vouz venez de trouver. Vous pouvez utilizer ce site: https://weblogo.berkeley.edu/logo.cgi. Affichez votre logo ci-dessous.

Motif LOGO:

10\. Avez-vous réussi à trouver votre motif initial implanté en séquences? l'algorithme était-il rapide? Quelle est la complexité de chaque recherche de motif?

C'est très lent si nous utilisons les kmers générés apr GenerateKmers(alphabet, k) car ici il y a plus de 200 000
combinaison possible, et l'algorithme du seed a une complité plutot elevé (mais bcp moins par rapport aux algorithme
des TMEs précédent !).
Cependant elle est assez rapide si nous générérons tous les kmers possible directement depuis les séquences (avec la fonction : tousLesKmers(sequences_implanted_with_var, k) car il y a environ 3900 kmers possible ici, en enlevant les kmers non-complexe !

11\. Tester l'algorithme  suffix tree sur vos données de chipSeq. Puis générér le LOGO du motif trouvé

In [ ]:
minrep = 5

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    #sequence = [sequenceStr]
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    return sequence[:len(sequence) - 2]
genome = "Sequence_by_Peaks_8.fasta" #votre fichier
sequences_chips = readFasta(genome,n)
#print(sequences)

print("Execution")
kmers3 = tousLesKmers(sequences_chips, k)
kmersValid3 = removeLowComplexe(kmers3, minrep)
kmersValid3 = removeLowComplexePair(kmersValid3)
print(len(kmersValid3))
tree4 = construct_tree(sequences_chips)

test_chips = inexact_match(kmersValid3, sequences_chips, tree4, v)
print(list(test_chips.items())[0])

mot2,(seq_tmp2, nbVa2r) = list(test_chips.items())[0]
file2 = open("FichierPourLogo2.txt", "w")
for s in seq_tmp2:
    file2.write(s)
    file2.write("\n")
file2.close()
#Ca met trop de temps....
print("Ca passe")

Execution
39157


12\. Créez le motif logo à partir des séquences du meilleur motif variable que vouz venez de trouver. Vous pouvez utilizer ce site: https://weblogo.berkeley.edu/logo.cgi. Affichez votre logo ci-dessous.
